In [3]:
# 1. Install necessary libraries
# We use 'quiet' to keep the output clean, remove it if you want to see installation details
!pip install pandas nltk textblob --quiet

# 2. Import modules
import pandas as pd
import random
import json
from datetime import datetime
import nltk
from textblob import TextBlob # Good for simple sentiment detection (keeping mood positive)

# 3. Download required NLTK data for tokenizing text
nltk.download('punkt', quiet=True)
nltk.download('brown', quiet=True) # Useful corpus for reference

# 4. Define a simple configuration for the bot
BOT_CONFIG = {
    "bot_name": "Remi", # Short for Reminiscence
    "user_name": "User", # We can change this dynamically later
    "session_start": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
}

print(f"✅ Environment Ready for {BOT_CONFIG['bot_name']}!")
print(f"Session started at: {BOT_CONFIG['session_start']}")

✅ Environment Ready for Remi!
Session started at: 2026-02-09 22:27:28


From the code above, 

pandas: We will use this to save conversation logs (very important for analysis later).

textblob: A simple library to detect sentiment. If a user gets agitated (negative sentiment), we can program the bot to change the subject.

BOT_CONFIG: A dictionary to store basic settings we can reference later.

We now want to move to Step 2: Defining the "Reminiscence Knowledge Base" This is the brain the bot will draw from.

In [4]:
# Define the topics and specific reminiscence prompts
reminiscence_topics = {
    "childhood": [
        "What was your favorite game to play outside when you were young?",
        "Do you remember the house you grew up in? What was your favorite room?",
        "What was your favorite home-cooked meal as a child?",
        "Did you have any pets growing up? What were their names?"
    ],
    "career": [
        "What was your very first job? Did you enjoy it?",
        "What is a professional accomplishment you are most proud of?",
        "Who was the best boss or mentor you ever had?",
        "How did your work change over the years?"
    ],
    "hobbies": [
        "What hobbies have you enjoyed throughout your life?",
        "Is there a particular song or type of music that always makes you smile?",
        "Did you enjoy traveling? Where was your favorite place to visit?",
        "What is a book or movie that had a big impact on you?"
    ],
    "family": [
        "How did you meet your spouse or partner?",
        "What is a favorite holiday tradition your family had?",
        "Tell me about a memorable family vacation.",
        "What values were most important to your parents?"
    ]
}

# Simple function to get a random question from a topic
def get_prompt(topic):
    if topic in reminiscence_topics:
        return random.choice(reminiscence_topics[topic])
    else:
        return "I'd love to hear more about that. Can you tell me more?"

print("✅ Knowledge Base Loaded with topics:", list(reminiscence_topics.keys()))

✅ Knowledge Base Loaded with topics: ['childhood', 'career', 'hobbies', 'family']


We are keeping the conversation focused on therapeutic areas and later we can add other information like music, school, e.t.c.

Defining The Logic Functions:

Next, we are going to build the "Brain" It is a function that takes what the user says, analyzes it and decides the memory topic to trigger next.

In [5]:
# --- REMI V6.1: Softer Empathy Layer ---

import random
from textblob import TextBlob

# Emotion-aware response pools (less clinical, more human)
empathy_responses = {
    "bad": [
        "I'm sorry today feels bad.",
        "That sounds like a hard day.",
        "Some days are just heavy."
    ],
    "sad": [
        "That sounds really sad.",
        "I'm here with you.",
        "I'm glad you told me."
    ],
    "tired": [
        "Feeling tired can be exhausting.",
        "Maybe your body needs some rest.",
        "We can take things slowly."
    ],
    "lonely": [
        "Feeling lonely is really hard.",
        "You're not alone right now.",
        "I'm here with you."
    ],
    "angry": [
        "That sounds frustrating.",
        "It's okay to feel upset.",
        "I hear how strong that feeling is."
    ],
    "anxious": [
        "It's okay. You're safe right now.",
        "Let's slow down together.",
        "Nothing is urgent."
    ]
}

grounding_phrases = [
    "You're safe right now.",
    "Nothing needs to be done.",
    "We can just sit for a moment."
]

chat_state = {
    "current_topic": None,
    "asked_questions": set(),
    "standing_by": False,
    "comfort_mode": False
}

print("✅ Remi V6.1 Empathy Layer Loaded")


✅ Remi V6.1 Empathy Layer Loaded


In [ ]:
# --- REMI V7: The Active Listener ---

import random
from textblob import TextBlob

# 1. CONTEXTUAL RESPONSES (Specific triggers for specific words)
context_replies = {
    "soup": "Soup sounds warm and comforting. Do you have a favorite kind, like tomato or chicken noodle?",
    "sandwich": "A sandwich is a good idea. simple and filling.",
    "water": "Sipping some water can really help clarify things.",
    "tea": "A hot cup of tea might be nice right now.",
    "coffee": "The smell of coffee is always so waking, isn't it?",
    "tired": "If you are tired, maybe we can just sit quietly. You don't have to talk.",
    "bed": "Resting in bed sounds like the best place to be right now."
}

chat_state = {
    "standing_by": False,
    "last_topic": None
}

def get_active_response(user_input):
    text = user_input.lower().strip()
    
    # --- LAYER 1: STRICT SILENCE (The "Go Away" Fix) ---
    # If the bot is in standby, it ONLY wakes up for specific "safe words"
    if chat_state["standing_by"]:
        if text in ["hello", "hi", "remi", "help", "start"]:
            chat_state["standing_by"] = False
            return "I am here. I was waiting for you."
        else:
            return "..." # IT WILL ACTUALLY PRINT DOTS AND DO NOTHING ELSE

    # --- LAYER 2: SHUTDOWN TRIGGERS ---
    if any(w in text for w in ["go away", "leave me", "stop talking", "shut up", "hate you"]):
        chat_state["standing_by"] = True
        return "I understand. I will be silent now. Say 'hello' if you want me back."

    # --- LAYER 3: SAFETY & CRISIS (The "I am not safe" Fix) ---
    if "not safe" in text or "scared" in text or "afraid" in text:
        return "That sounds frightening. Are you alone right now? Is there someone real you can call?"

    if "upset" in text or "angry" in text:
        return "I can hear that you are upset. I am listening. Tell me what is making you feel this way."

    # --- LAYER 4: PHYSICAL NEEDS (The "Soup" Fix) ---
    # Check if the user mentioned a specific item we know about
    for keyword, reply in context_replies.items():
        if keyword in text:
            return reply
            
    if any(w in text for w in ["hungry", "food", "eat"]):
        return "You mentioned being hungry. Do you have any food nearby, maybe some soup or bread?"

    # --- LAYER 5: DIRECT QUESTIONS (The "How are you" Fix) ---
    if "how are you" in text:
        return "I am just a computer program, so I don't have feelings, but I function better when I can help you. How are YOU feeling?"

    if "who are you" in text:
        return "I am Remi. I'm here to keep you company and chat about whatever you want."

    # --- LAYER 6: SIMPLE REFLECTION (Better than "I'm here") ---
    # If the user says a short sentence, mirror it back or ask an open question
    if text.startswith("i am"):
        # e.g., "I am cold" -> "You are cold? Do you have a blanket nearby?"
        return f"You are feeling {text.replace('i am', '').strip()}? Tell me more about that."

    if text in ["no", "nope", "nothing"]:
        return "Okay. We can just sit here. You don't have to say anything."

    if text in ["yes", "yeah"]:
        return "I'm listening."

    # Fallback to general conversation if nothing else matches
    return "I hear you. What else is on your mind?"

# -----------------------------
# INTERACTIVE LOOP
# -----------------------------
print("--- Remi V7 (Active Listener) ---")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit"]:
        print("Remi: Goodbye.")
        break
        
    response = get_active_response(user_input)
    print(f"Remi: {response}")

--- Remi V7 (Active Listener) ---


You:  Good morning?


Remi: I hear you. What else is on your mind?


You:  I am hungry


Remi: You mentioned being hungry. Do you have any food nearby, maybe some soup or bread?


You:  no


Remi: Okay. We can just sit here. You don't have to say anything.


You:  why?


Remi: I hear you. What else is on your mind?


The above is important because of context awareness, i.e if the user says I loved my mum, the bot will detect family and ask a family question next. It acts as a safety valve i.e < -0.5 check if the patient gets upset, the bot tries to pivot to a happier subject(music/hobbies)

we are now going to stitch all those pieces together into a running loop. This will be the main chat loop.

In [ ]:
I am not aware of what to do next